In [4]:
# ============================
# Projeto Aplicado III – Etapa 2
# Prova de Conceito – Sistema de Recomendação
# ============================

# Importando bibliotecas necessárias
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Definindo caminho da base limpa
PATH_CLEAN = "/Users/iloop/Desktop/Projeto/udemy_cleaned_for_training.csv"

# Carregando dataset tratado
df = pd.read_csv(PATH_CLEAN)

# Conferindo colunas
df.head()


,course_title,price,num_subscribers,level_52,level_All Levels,level_Beginner Level,level_Expert Level,level_Intermediate Level,subject_Business Finance,subject_Graphic Design,subject_Musical Instruments,subject_Web Development,num_subscribers_log,price_norm
0,ultimate investment banking course,200,2147,False,True,False,False,False,True,False,False,False,7.672292,1.000
1,complete gst course certification grow your ca...,75,2792,False,True,False,False,False,True,False,False,False,7.934872,0.375
2,financial modeling for business analysts and c...,45,2174,False,False,False,False,True,True,False,False,False,7.684784,0.225
3,beginner to pro financial analysis in excel 2017,95,2451,False,True,False,False,False,True,False,False,False,7.804659,0.475
4,how to maximize your profits trading options,200,1276,False,False,False,False,True,True,False,False,False,7.152269,1.000


In [5]:
# ============================
# 1. Criar matriz TF-IDF a partir do título dos cursos
# ============================

# Preenchendo valores nulos em 'course_title' com string vazia
df['course_title'] = df['course_title'].fillna("")

# Inicializando o vetorizador TF-IDF
tfidf = TfidfVectorizer(stop_words="english")

# Ajustando e transformando os títulos em vetores numéricos
tfidf_matrix = tfidf.fit_transform(df['course_title'])

# Conferindo dimensões da matriz
print("Matriz TF-IDF:", tfidf_matrix.shape)


Matriz TF-IDF: (3677, 3541)


In [6]:
# ============================
# 2. Calcular matriz de similaridade do cosseno
# ============================

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("Matriz de similaridade gerada com sucesso!")


Matriz de similaridade gerada com sucesso!


In [7]:
# ============================
# 3. Função para recomendar cursos
# ============================

def recomendar_cursos(titulo_curso, n_recomendacoes=5):
    # Localiza o índice do curso no dataset
    idx = df[df['course_title'].str.contains(titulo_curso, case=False, na=False)].index[0]

    # Obtém a lista de similaridades para esse curso
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordena pela similaridade (maior para menor)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Ignora o próprio curso (primeiro item) e pega os n recomendados
    sim_scores = sim_scores[1:n_recomendacoes+1]

    # Retorna títulos recomendados
    cursos_recomendados = [df.iloc[i[0]]['course_title'] for i in sim_scores]

    return cursos_recomendados


In [8]:
# ============================
# 4. Testando a recomendação
# ============================

curso_exemplo = "python"
print("Curso base:", curso_exemplo)
print("\nCursos recomendados:\n")

for c in recomendar_cursos(curso_exemplo, 5):
    print("-", c)


Curso base: python

Cursos recomendados:

- python algo stock trading automate your trading
- python algo trading fx trading with oanda
- black algo trading build your trading robot
- python for trading investing
- forex economic news trading
